### 노이즈 기반 종목 탐색 및 자동매매

In [3]:
from os import access
import time
from numpy import absolute
import pyupbit
import datetime
import pandas as pd

### 1단계 노이즈(1-abs(시가-종가)/고가-저가)가 0.4 이하인 종목 탐색

In [4]:
tickers = pyupbit.get_tickers(fiat="KRW")
print(tickers)

['KRW-BTC', 'KRW-ETH', 'KRW-NEO', 'KRW-MTL', 'KRW-LTC', 'KRW-XRP', 'KRW-ETC', 'KRW-OMG', 'KRW-SNT', 'KRW-WAVES', 'KRW-XEM', 'KRW-QTUM', 'KRW-LSK', 'KRW-STEEM', 'KRW-XLM', 'KRW-ARDR', 'KRW-ARK', 'KRW-STORJ', 'KRW-GRS', 'KRW-REP', 'KRW-ADA', 'KRW-SBD', 'KRW-POWR', 'KRW-BTG', 'KRW-ICX', 'KRW-EOS', 'KRW-TRX', 'KRW-SC', 'KRW-ONT', 'KRW-ZIL', 'KRW-POLY', 'KRW-ZRX', 'KRW-LOOM', 'KRW-BCH', 'KRW-BAT', 'KRW-IOST', 'KRW-RFR', 'KRW-CVC', 'KRW-IQ', 'KRW-IOTA', 'KRW-MFT', 'KRW-ONG', 'KRW-GAS', 'KRW-UPP', 'KRW-ELF', 'KRW-KNC', 'KRW-BSV', 'KRW-THETA', 'KRW-QKC', 'KRW-BTT', 'KRW-MOC', 'KRW-ENJ', 'KRW-TFUEL', 'KRW-MANA', 'KRW-ANKR', 'KRW-AERGO', 'KRW-ATOM', 'KRW-TT', 'KRW-CRE', 'KRW-MBL', 'KRW-WAXP', 'KRW-HBAR', 'KRW-MED', 'KRW-MLK', 'KRW-STPT', 'KRW-ORBS', 'KRW-VET', 'KRW-CHZ', 'KRW-STMX', 'KRW-DKA', 'KRW-HIVE', 'KRW-KAVA', 'KRW-AHT', 'KRW-LINK', 'KRW-XTZ', 'KRW-BORA', 'KRW-JST', 'KRW-CRO', 'KRW-TON', 'KRW-SXP', 'KRW-HUNT', 'KRW-PLA', 'KRW-DOT', 'KRW-SRM', 'KRW-MVL', 'KRW-STRAX', 'KRW-AQT', 'KRW-GLM', 

In [5]:
df = pd.DataFrame()
for ticker in tickers:
    try:
        temp = pyupbit.get_ohlcv(ticker, interval="day", count=20)
        temp["ticker"] = ticker
        df = pd.concat([df, temp])
    except:
        pass
    
df.head()

,open,high,low,close,volume,value,ticker
2021-11-12 09:00:00,78938000.0,79100000.0,76712000.0,78165000.0,7117.165348,5.555562e+11,KRW-BTC
2021-11-13 09:00:00,78165000.0,78770000.0,77120000.0,77867000.0,4350.104533,3.385629e+11,KRW-BTC
2021-11-14 09:00:00,77867000.0,79350000.0,77579000.0,79310000.0,3826.416588,3.001073e+11,KRW-BTC
2021-11-15 09:00:00,79323000.0,80250000.0,77330000.0,77602000.0,6972.545226,5.506619e+11,KRW-BTC
2021-11-16 09:00:00,77600000.0,77602000.0,73325000.0,73897000.0,12978.656081,9.728458e+11,KRW-BTC


In [6]:
df_noise = pd.DataFrame()
for ticker in tickers:
    try:
        temp = df
        temp["noise"] = 1 - (absolute(df["open"] - df["close"]) / absolute(df["high"] - df["low"]))
        df_noise= pd.concat([df, temp])
    except:
        pass
    
df_noise.head()

,open,high,low,close,volume,value,ticker,noise
2021-11-12 09:00:00,78938000.0,79100000.0,76712000.0,78165000.0,7117.165348,5.555562e+11,KRW-BTC,0.676298
2021-11-13 09:00:00,78165000.0,78770000.0,77120000.0,77867000.0,4350.104533,3.385629e+11,KRW-BTC,0.819394
2021-11-14 09:00:00,77867000.0,79350000.0,77579000.0,79310000.0,3826.416588,3.001073e+11,KRW-BTC,0.185206
2021-11-15 09:00:00,79323000.0,80250000.0,77330000.0,77602000.0,6972.545226,5.506619e+11,KRW-BTC,0.410616
2021-11-16 09:00:00,77600000.0,77602000.0,73325000.0,73897000.0,12978.656081,9.728458e+11,KRW-BTC,0.134206


### avgnoise가 0.5아래인 ticker 리스트 추출(tiker이름, avgnoise)

In [8]:
check_list1 = []

for ticker in tickers :
    check = df_noise[df_noise['ticker']==ticker]
    print(ticker, check['noise'].mean())
    if check['noise'].mean() < 0.5 :
        print(ticker, check['noise'].mean())
        check_list1.append(ticker)

print(check_list1)


KRW-BTC 0.49380489132333005
KRW-BTC 0.49380489132333005
KRW-ETH 0.459374296289018
KRW-ETH 0.459374296289018
KRW-NEO 0.554138240587033
KRW-MTL 0.5971592192081365
KRW-LTC 0.5306148887670743
KRW-XRP 0.6402548863971278
KRW-ETC 0.6136669138030777
KRW-OMG 0.42531384558163515
KRW-OMG 0.42531384558163515
KRW-SNT 0.5677579365079365
KRW-WAVES 0.7073399770447629
KRW-XEM 0.564589972935561
KRW-QTUM 0.5956670378089365
KRW-LSK 0.592752844888582
KRW-STEEM nan
KRW-XLM nan
KRW-ARDR nan
KRW-ARK nan
KRW-STORJ 0.601496561116037
KRW-GRS 0.5770683733593327
KRW-REP 0.497809591742322
KRW-REP 0.497809591742322
KRW-ADA 0.5512844266336911
KRW-SBD 0.6080538221650242
KRW-POWR 0.5724617118221537
KRW-BTG 0.6304590461074409
KRW-ICX 0.55185047177754
KRW-EOS 0.5651361516757815
KRW-TRX 0.4991818086883876
KRW-TRX 0.4991818086883876
KRW-SC nan
KRW-ONT nan
KRW-ZIL nan
KRW-POLY nan
KRW-ZRX 0.5761987373231883
KRW-LOOM 0.5844302213214664
KRW-BCH 0.494111431363318
KRW-BCH 0.494111431363318
KRW-BAT 0.574758249541248
KRW-IOST 0.5

## 2단계: 가격 상승 + 거래량 상승 코인 찾기(5일 이동평균 이용)

### 상승장 파악

In [9]:
df = pd.DataFrame()
for ticker in check_list1:
    try:
        temp = pyupbit.get_ohlcv(ticker, interval="day", count=5)
        temp["ticker"] = ticker
        df = pd.concat([df, temp])
    except:
        pass
    
df.head(10)

,open,high,low,close,volume,value,ticker
2021-11-27 09:00:00,68550000.0,70276000.0,68140000.0,70039000.0,4865.737853,3.383817e+11,KRW-BTC
2021-11-28 09:00:00,70039000.0,72691000.0,68607000.0,72303000.0,5439.530304,3.809438e+11,KRW-BTC
2021-11-29 09:00:00,72255000.0,73166000.0,71298000.0,72240000.0,5965.745427,4.312498e+11,KRW-BTC
2021-11-30 09:00:00,72241000.0,73300000.0,70000000.0,70702000.0,8717.765862,6.233846e+11,KRW-BTC
2021-12-01 09:00:00,70771000.0,72487000.0,70500000.0,72265000.0,5680.394766,4.033118e+11,KRW-BTC
2021-11-27 09:00:00,5151000.0,5318000.0,5115000.0,5249000.0,34072.648597,1.786141e+11,KRW-ETH
2021-11-28 09:00:00,5248000.0,5446000.0,5110000.0,5420000.0,47900.643372,2.509142e+11,KRW-ETH
2021-11-29 09:00:00,5420000.0,5567000.0,5370000.0,5551000.0,40742.686191,2.225906e+11,KRW-ETH
2021-11-30 09:00:00,5550000.0,5895000.0,5427000.0,5739000.0,92562.744308,5.264305e+11,KRW-ETH
2021-12-01 09:00:00,5739000.0,5900000.0,5702000.0,5856000.0,44530.290278,2.598338e+11,KRW-ETH
